In [49]:
import pandas as pd
import polars as pl

In [50]:
df = pl.read_json("ShareGPT_V3_unfiltered_cleaned_split_no_imsorry.json")

In [51]:
df_pd = pd.read_json("ShareGPT_V3_unfiltered_cleaned_split_no_imsorry.json")

In [52]:
df.estimated_size("mb")

585.2572727203369

In [53]:
df.shape

(94145, 2)

In [54]:
df = (
    # Take the first part of the struct as the questionner
    df.with_columns(pl.col("conversations").list.first().alias("human"))
    # Unnest the struct and rename it
    .unnest("human")
    .rename({"from": "questionner", "value": "question"})
    # Take the last part as the answerer
    .with_columns(pl.col("conversations").list.last().alias("gpt"))
    # unnest and rename it
    .unnest("gpt")
    .drop("conversations")
    .rename({"from": "answerer", "value": "answer"})
)
df.head()

id,questionner,question,answerer,answer
str,str,str,str,str
"""QWJhYvA_0""","""human""","""Summarize the …","""gpt""","""Tony Robbins d…"
"""i6IyJda_0""","""human""","""How to tell if…","""gpt""","""1. Homogeneity…"
"""A5AbcES_0""","""human""","""In Java, I wan…","""gpt""","""You can use th…"
"""hRPPgZT_0""","""human""","""Metaphorical l…","""gpt""","""Lo and behold!…"
"""hRPPgZT_11""","""gpt""","""Lo and behold!…","""gpt""","""Behold, fellow…"


In [55]:
df_pd["questionner"] = df_pd["conversations"].str.get(0).str.get("from")
df_pd["answerer"] = df_pd["conversations"].str.get(1).str.get("from")
df_pd["question"] = df_pd["conversations"].str.get(0).str.get("value")
df_pd["answer"] = df_pd["conversations"].str.get(1).str.get("value")
df_pd = df_pd.drop(columns="conversations")
df_pd.head()

,id,questionner,answerer,question,answer
0,QWJhYvA_0,human,gpt,Summarize the main ideas of Jeff Walker's Prod...,Here are the main ideas of Jeff Walker's Produ...
1,i6IyJda_0,human,gpt,How to tell if a customer segment is well segm...,1. Homogeneity: The segment should consist of ...
2,A5AbcES_0,human,gpt,"In Java, I want to replace string like ""This i...",You can use the `String.format()` method in Ja...
3,hRPPgZT_0,human,gpt,Metaphorical language is also used to describe...,Metaphorical language has been summoned to des...
4,hRPPgZT_11,gpt,human,Lo and behold! By the grace of divine interven...,More verbose and details


In [56]:
df = (
    # Just take questions under 5 words
    df.with_columns(pl.col("question").str.split(" ").alias("number"))
    .filter(pl.col("number").list.len() < 5)
    .drop("number")
    .with_columns(
        pl.col("question", "answer")
        # Replace everything that is not alpha-numeric by spaces and drop nulls
        .str.replace_all(r"[^a-zA-Z]", " ")
    )
    .drop_nulls()
)
df.head()

id,questionner,question,answerer,answer
str,str,str,str,str
"""VY7cMKG_0""","""human""","""Poly Ether Eth…","""gpt""","""Here are the t…"
"""ng7rjf6_0""","""human""","""explane more""","""gpt""","""Sure Here s a…"
"""W4wL13P_0""","""human""","""explain the me…","""gpt""","""The metaverse …"
"""v4PzAY8_0""","""human""","""Hello""","""gpt""","""Hello How can…"
"""mxEe31E_0""","""human""","""Explain deposi…","""gpt""","""Deposition and…"


In [57]:
df_pd["number"] = df_pd["question"].str.split(" ")
df_pd = df_pd[
    df_pd["number"].apply(lambda x: len(x) < 5 if isinstance(x, list) else False)
]
df_pd = df_pd.drop(columns=["number"])

# Replace non-alphabetic characters in 'question' and 'answer' columns and drop nulls
df_pd["question"] = df_pd["question"].str.replace(r"[^a-zA-Z]", " ", regex=True)
df_pd["answer"] = df_pd["answer"].str.replace(r"[^a-zA-Z]", " ", regex=True)

df_bis = df_pd.dropna()
df_bis.head()

,id,questionner,answerer,question,answer
9,VY7cMKG_0,human,gpt,Poly Ether Ether Ketone,Poly Ether Ether Ketone PEEK is a high perfo...
19,ng7rjf6_0,human,gpt,explane more,Sure I d be happy to explain in more detail ...
68,W4wL13P_0,human,gpt,explain the metaverse,The metaverse is a term used to describe a col...
86,v4PzAY8_0,human,gpt,Hello,Hello How can I help you today
91,mxEe31E_0,human,gpt,Explain deposition,Deposition is a process in which material or p...
